In [ ]:
import numpy as np
import netCDF4 as nc
import time, os
from glob import glob

In [ ]:
def prepare_nc(input_path, output_path):
    """
    Takes ResNet-50 network predictions as input, 
    produces netCDF4 file with a given output path and name.
    """
    
    # Open data and specify output netCDF4 file.
    raw_data = np.load(input_path)
    for c in xrange(raw_data.shape[2]):
        chan = raw_data[:, :, c].copy()
        min_chan = np.min(chan)
        chan = chan - min_chan
        max_chan = np.max(chan)
        chan = chan / max_chan
        raw_data[:, :, c] = chan
    
    dataset = nc.Dataset(output_path, 'w')
    
    # Create dimensions.
    lat = dataset.createDimension('lat', raw_data.shape[0])
    lon = dataset.createDimension('lon', raw_data.shape[1])
    level = dataset.createDimension('level', raw_data.shape[2])
    
    # Create variables.
    lats = dataset.createVariable('lat', np.float32, ('lat',)) 
    lons = dataset.createVariable('lon', np.float32, ('lon',)) 
    levels = dataset.createVariable('level', np.float32, ('level',))
    
    # Create 3D data.
    temp = dataset.createVariable('temp', np.float32, ('lat','lon','level'))
    dataset.description = 'ResNet-50 predictions dataset'
    dataset.history = 'Created ' + time.ctime(time.time())
    dataset.source = 'ResNet-50, Microsoft'
    
    # Assigning data.
    lats[:] = np.arange(raw_data.shape[0] - 1, -1, -1)
    lons[:] = np.arange(raw_data.shape[1] - 1, -1, -1)
    levels[:] = np.arange(0, raw_data.shape[2],1)
    temp[:] = raw_data
    
    # Close dataset and save it to netCDF4 file.
    dataset.close()

    

In [ ]:
raw_datas = glob('./outputs-resized-single-filter/*/*/data.npy')
try:
    os.mkdir("netcdf")
except:
    pass

for raw_data in raw_datas:
    input_path = raw_data
    label = raw_data.split("/")[-3]
    img_name = raw_data.split("/")[-2]
    
    try:
        os.mkdir("netcdf/{}".format(label))
    except:
        pass
    try:
        os.mkdir("netcdf/{}/{}".format(label, img_name))
    except:
        pass
    
    output_path = "netcdf/{}/{}/dataset.nc".format(label, img_name)
        
    prepare_nc(input_path, output_path)